# MecaChess Robotic Arm - Final Integration (Raspberry Pi Pico)
Complete autonomous chess playing robotic arm system

## System Configuration and Imports

In [ ]:
import cv2
import numpy as np
import serial
import time
import math
from math import radians, degrees, cos, sin, atan2, sqrt, acos
import threading
from datetime import datetime

# System Configuration
class SystemConfig:
    # Pico Serial
    PICO_PORT = 'COM15'
    PICO_BAUD = 115200
    
    # Arm Parameters (mm)
    BASE_HEIGHT = 96.0
    BASE_OFFSET = 13.9
    LINK1_LENGTH = 150.0
    LINK2_LENGTH = 143.24
    END_EFFECTOR = 127.42
    
    # Board Detection
    BOARD_SQUARES = 8
    SQUARE_SIZE_MM = 35
    BOARD_START_X = 100
    BOARD_START_Y = 100
    
    # Camera
    CAMERA_URL = "http://10.81.34.95:8080/video"
    
# System state
system_state = {
    'connected': False,
    'last_move': None,
    'board_position': None,
    'executing_move': False
}

pico_connection = None

## Pico Communication Module

In [ ]:
class PicoController:
    def __init__(self, port, baud_rate):
        self.port = port
        self.baud_rate = baud_rate
        self.connection = None
    
    def connect(self):
        """Establish connection to Pico"""
        try:
            self.connection = serial.Serial(self.port, self.baud_rate, timeout=2)
            time.sleep(2)  # Wait for Pico to initialize
            print(f"[{datetime.now()}] Connected to Pico on {self.port}")
            system_state['connected'] = True
            return True
        except Exception as e:
            print(f"Connection error: {e}")
            return False
    
    def disconnect(self):
        """Close connection to Pico"""
        if self.connection:
            self.connection.close()
            system_state['connected'] = False
            print(f"[{datetime.now()}] Disconnected from Pico")
    
    def send_move(self, from_square, to_square, is_capture=False):
        """Send chess move to Pico"""
        if not self.connection or not system_state['connected']:
            print("Error: Not connected to Pico")
            return False
        
        try:
            move_type = "1" if is_capture else "0"
            message = f"{from_square} {to_square} {move_type}\n"
            
            system_state['executing_move'] = True
            self.connection.write(message.encode())
            print(f"[{datetime.now()}] Sent move: {message.strip()}")
            
            # Wait for acknowledgment
            response = self.connection.readline().decode().strip()
            if response:
                print(f"Pico response: {response}")
            
            system_state['last_move'] = {'from': from_square, 'to': to_square}
            system_state['executing_move'] = False
            return True
        except Exception as e:
            print(f"Send error: {e}")
            system_state['executing_move'] = False
            return False
    
    def send_angles(self, q1, q2, q3, q4):
        """Send joint angles directly to Pico"""
        if not self.connection or not system_state['connected']:
            print("Error: Not connected to Pico")
            return False
        
        try:
            message = f"ANGLES {q1:.2f} {q2:.2f} {q3:.2f} {q4:.2f}\n"
            self.connection.write(message.encode())
            print(f"[{datetime.now()}] Sent angles: {message.strip()}")
            return True
        except Exception as e:
            print(f"Send error: {e}")
            return False

## Kinematics Module

In [ ]:
class Kinematics:
    @staticmethod
    def forward(q1, q2, q3, q4):
        """Calculate end-effector position from joint angles"""
        q1r = radians(q1)
        q2r = radians(q2)
        q3r = radians(q3)
        q4r = radians(q4)
        
        # Position calculation
        r = (SystemConfig.LINK1_LENGTH * cos(q2r) + 
             SystemConfig.LINK2_LENGTH * cos(q2r + q3r) + 
             SystemConfig.END_EFFECTOR * cos(q2r + q3r + q4r))
        
        z = (SystemConfig.BASE_HEIGHT + 
             SystemConfig.LINK1_LENGTH * sin(q2r) + 
             SystemConfig.LINK2_LENGTH * sin(q2r + q3r) + 
             SystemConfig.END_EFFECTOR * sin(q2r + q3r + q4r))
        
        x = r * cos(q1r)
        y = r * sin(q1r)
        
        return x, y, z
    
    @staticmethod
    def inverse(x, y, z):
        """Calculate joint angles from target position"""
        try:
            q1 = atan2(y, x)
            q1_deg = degrees(q1)
            
            r = sqrt(x**2 + y**2)
            z_work = z - SystemConfig.BASE_HEIGHT
            distance = sqrt(r**2 + z_work**2)
            
            l34_total = SystemConfig.LINK2_LENGTH + SystemConfig.END_EFFECTOR
            cos_q3 = (distance**2 - SystemConfig.LINK1_LENGTH**2 - l34_total**2) / (2 * SystemConfig.LINK1_LENGTH * l34_total)
            
            if abs(cos_q3) > 1:
                return None  # Unreachable
            
            q3_rad = acos(cos_q3)
            q3_deg = degrees(q3_rad) - 180
            
            alpha = atan2(z_work, r)
            beta = atan2(l34_total * sin(q3_rad), SystemConfig.LINK1_LENGTH + l34_total * cos(q3_rad))
            q2_deg = degrees(alpha) + degrees(beta)
            
            q4_deg = -q2_deg - q3_deg
            
            return q1_deg, q2_deg, q3_deg, q4_deg
        except Exception as e:
            print(f"Inverse kinematics error: {e}")
            return None

## Chess Board Detection Module

In [ ]:
class ChessBoardDetection:
    @staticmethod
    def capture_board_image(camera_url=None):
        """Capture chess board image from camera"""
        try:
            if camera_url:
                cap = cv2.VideoCapture(camera_url)
            else:
                cap = cv2.VideoCapture(0)  # Use local camera
            
            time.sleep(2)  # Wait for camera
            ret, frame = cap.read()
            
            if ret:
                cv2.imwrite("board_snapshot.jpg", frame)
                print("Board image captured")
                cap.release()
                return True
            else:
                print("Failed to capture image")
                cap.release()
                return False
        except Exception as e:
            print(f"Camera error: {e}")
            return False
    
    @staticmethod
    def square_to_coordinates(square):
        """Convert chess square notation to physical coordinates"""
        files = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
        ranks = {'1': 7, '2': 6, '3': 5, '4': 4, '5': 3, '6': 2, '7': 1, '8': 0}
        
        file_idx = files[square[0]]
        rank_idx = ranks[square[1]]
        
        x = SystemConfig.BOARD_START_X + (file_idx * SystemConfig.SQUARE_SIZE_MM)
        y = SystemConfig.BOARD_START_Y + (rank_idx * SystemConfig.SQUARE_SIZE_MM)
        
        return x, y

## Main Control System

In [ ]:
class MecaChessController:
    def __init__(self):
        self.pico = PicoController(SystemConfig.PICO_PORT, SystemConfig.PICO_BAUD)
        self.kinematics = Kinematics()
        self.board_detection = ChessBoardDetection()
    
    def initialize_system(self):
        """Initialize the entire system"""
        print("="*60)
        print("MecaChess Robotic Arm - Raspberry Pi Pico Edition")
        print("="*60)
        
        # Connect to Pico
        if not self.pico.connect():
            print("Cannot proceed without Pico connection")
            return False
        
        print("System initialized successfully")
        return True
    
    def move_piece(self, from_square, to_square, is_capture=False):
        """Execute a chess move"""
        print(f"\nExecuting move: {from_square} -> {to_square}")
        
        # Get coordinates
        from_x, from_y = self.board_detection.square_to_coordinates(from_square)
        to_x, to_y = self.board_detection.square_to_coordinates(to_square)
        
        z_pick = 50  # Height above board for picking
        z_place = 50  # Height above board for placing
        
        # Calculate angles for pick position
        pick_angles = self.kinematics.inverse(from_x, from_y, z_pick)
        if pick_angles:
            self.pico.send_angles(*pick_angles)
            time.sleep(1)
        
        # Calculate angles for place position
        place_angles = self.kinematics.inverse(to_x, to_y, z_place)
        if place_angles:
            self.pico.send_angles(*place_angles)
            time.sleep(1)
        
        # Send move command
        self.pico.send_move(from_square, to_square, is_capture)
        
        print(f"Move completed")
    
    def shutdown(self):
        """Shutdown the system"""
        self.pico.disconnect()
        print("System shutdown complete")

# Create global controller instance
controller = MecaChessController()

## Example Usage and Testing

In [ ]:
# Initialize system
if controller.initialize_system():
    print("\nSystem ready for moves")
    print("Example: controller.move_piece('e2', 'e4')")
    
    # Example moves (uncomment to execute)
    # controller.move_piece('e2', 'e4')  # Standard opening
    # controller.move_piece('g1', 'f3')  # Knight move
    
else:
    print("Failed to initialize system")

# To shutdown
# controller.shutdown()

## System Status and Diagnostics

In [ ]:
def print_system_status():
    """Print current system status"""
    print("\n" + "="*60)
    print("MecaChess System Status")
    print("="*60)
    print(f"Connected to Pico: {system_state['connected']}")
    print(f"Last Move: {system_state['last_move']}")
    print(f"Executing: {system_state['executing_move']}")
    print(f"Board Position: {system_state['board_position']}")
    print("="*60 + "\n")

def test_kinematics():
    """Test kinematics calculations"""
    print("\nKinematics Test:")
    q1, q2, q3, q4 = 90, 45, -30, -60
    x, y, z = Kinematics.forward(q1, q2, q3, q4)
    print(f"Forward (Q1={q1}, Q2={q2}, Q3={q3}, Q4={q4}): X={x:.2f}, Y={y:.2f}, Z={z:.2f}")
    
    angles = Kinematics.inverse(x, y, z)
    if angles:
        print(f"Inverse (X={x:.2f}, Y={y:.2f}, Z={z:.2f}): Q1={angles[0]:.2f}, Q2={angles[1]:.2f}, Q3={angles[2]:.2f}, Q4={angles[3]:.2f}")

# Uncomment to run diagnostics
# print_system_status()
# test_kinematics()